## 循环和递归神经网络 Task8

* RNN的结构。循环神经网络的提出背景、优缺点。着重学习RNN的反向传播、RNN出现的问题（梯度问题、长期依赖问题）、BPTT算法。
* 双向RNN
* 递归神经网络
* LSTM、GRU的结构、提出背景、优缺点。
* 针对梯度消失（LSTM等其他门控RNN）、梯度爆炸（梯度截断）的解决方案。
* Memory Network（自选）
* Text-RNN的原理。
* 利用Text-RNN模型来进行文本分类。
* Recurrent Convolutional Neural Networks（RCNN）原理。
* 利用RCNN模型来进行文本分类。
* 参考：[一份详细的LSTM和GRU图解](https://www.atyun.com/30234.html) [Tensorflow实战(1): 实现深层循环神经网络](https://zhuanlan.zhihu.com/p/37070414) [lstm](https://x-algo.cn/index.php/2017/01/13/1609/) [RCNN kreas](https://github.com/airalcorn2/Recurrent-Convolutional-Neural-Network-Text-Classifier) [RCNN tf](https://github.com/zhangfazhan/TextRCNN) [RCNN tf 推荐](https://github.com/roomylee/rcnn-text-classification)


In [1]:
'''
* 利用Text-RNN模型来进行文本分类。
'''
# -*- coding: utf-8 -*-
#TextRNN: 1. embeddding输入层, 2.Bi-LSTM 层, 3.concat output连接输出层, 4.FC 全连接层, 5.softmax
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

class TextRNN:
    def __init__(self,num_classes, learning_rate, batch_size, decay_steps, decay_rate,sequence_length,
                 vocab_size,embed_size,is_training,initializer=tf.random_normal_initializer(stddev=0.1)):
       
        # 设置超参数
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.sequence_length=sequence_length
        self.vocab_size=vocab_size
        self.embed_size=embed_size
        self.hidden_size=embed_size
        self.is_training=is_training
        self.learning_rate=learning_rate
        self.initializer=initializer
        self.num_sampled=20

        # 初始化x,label,dropout
        self.input_x = tf.placeholder(tf.int32, [None, self.sequence_length], name="input_x")  # X
        self.input_y = tf.placeholder(tf.int32,[None], name="input_y")  # y [None,num_classes]
        self.dropout_keep_prob=tf.placeholder(tf.float32,name="dropout_keep_prob")

        self.global_step = tf.Variable(0, trainable=False, name="Global_Step")
        self.epoch_step=tf.Variable(0,trainable=False,name="Epoch_Step")
        self.epoch_increment=tf.assign(self.epoch_step,tf.add(self.epoch_step,tf.constant(1)))
        self.decay_steps, self.decay_rate = decay_steps, decay_rate

        self.instantiate_weights()
        self.logits = self.inference() #[None, self.label_size]. main computation graph is here.
        if not is_training:
            return
        self.loss_val = self.loss() #-->self.loss_nce()
        self.train_op = self.train()
        self.predictions = tf.argmax(self.logits, axis=1, name="predictions")  # shape:[None,]
        correct_prediction = tf.equal(tf.cast(self.predictions,tf.int32), self.input_y) #tf.argmax(self.logits, 1)-->[batch_size]
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="Accuracy") # shape=()
    def instantiate_weights(self):
        
        with tf.name_scope("embedding"): # embedding matrix
            self.Embedding = tf.get_variable("Embedding",shape=[self.vocab_size, self.embed_size],initializer=self.initializer) #[vocab_size,embed_size] tf.random_uniform([self.vocab_size, self.embed_size],-1.0,1.0)
            self.W_projection = tf.get_variable("W_projection",shape=[self.hidden_size*2, self.num_classes],initializer=self.initializer) #[embed_size,label_size]
            self.b_projection = tf.get_variable("b_projection",shape=[self.num_classes])       #[label_size]

    def inference(self):
       
        #1.【提取输入句中的词向量用tf的embedding_lookup】
      
        self.embedded_words = tf.nn.embedding_lookup(self.Embedding,self.input_x) #shape:[None,sentence_length,embed_size]
        
        #2. Bi-lstm 层
        # 定义 lstm 单元，取lstm单元输出【BasicLSTMCell】
        lstm_fw_cell=rnn.BasicLSTMCell(self.hidden_size) #前向单元
        lstm_bw_cell=rnn.BasicLSTMCell(self.hidden_size) #后向单元
        if self.dropout_keep_prob is not None:
            lstm_fw_cell=rnn.DropoutWrapper(lstm_fw_cell,output_keep_prob=self.dropout_keep_prob)
            lstm_bw_cell=rnn.DropoutWrapper(lstm_bw_cell,output_keep_prob=self.dropout_keep_prob)
        # bidirectional_dynamic_rnn: input: [batch_size, max_time, input_size]
        #                            output: A tuple (outputs, output_states)
        #                                    where:outputs: A tuple (output_fw, output_bw) containing the forward and the backward rnn output `Tensor`.
        outputs,_=tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell,lstm_bw_cell,self.embedded_words,dtype=tf.float32) #[batch_size,sequence_length,hidden_size] #creates a dynamic bidirectional recurrent neural network
        print("outputs:===>",outputs) #outputs:(<tf.Tensor 'bidirectional_rnn/fw/fw/transpose:0' shape=(?, 5, 100) dtype=float32>, <tf.Tensor 'ReverseV2:0' shape=(?, 5, 100) dtype=float32>))
        output_rnn=tf.concat(outputs,axis=2) #[batch_size,sequence_length,hidden_size*2]

        #3. 第二层 LSTM 
        rnn_cell=rnn.BasicLSTMCell(self.hidden_size*2)
        if self.dropout_keep_prob is not None:
            rnn_cell=rnn.DropoutWrapper(rnn_cell,output_keep_prob=self.dropout_keep_prob)
        _,final_state_c_h=tf.nn.dynamic_rnn(rnn_cell,output_rnn,dtype=tf.float32)
        final_state=final_state_c_h[1]

        #4 .FC全连接层
        output=tf.layers.dense(final_state,self.hidden_size*2,activation=tf.nn.tanh)
        
        #5. wx+b
        with tf.name_scope("output"): #inputs: A `Tensor` of shape `[batch_size, dim]`.  The forward activations of the input network.
            logits = tf.matmul(output, self.W_projection) + self.b_projection  # [batch_size,num_classes]
        return logits

    #损失函数
    def loss(self,l2_lambda=0.0001):
        with tf.name_scope("loss"):
            #input: `logits` and `labels` must have the same shape `[batch_size, num_classes]`
            #output: A 1-D `Tensor` of length `batch_size` of the same type as `logits` with the softmax cross entropy loss.
            losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.input_y, logits=self.logits);#sigmoid_cross_entropy_with_logits.#losses=tf.nn.softmax_cross_entropy_with_logits(labels=self.input_y,logits=self.logits)
            #print("1.sparse_softmax_cross_entropy_with_logits.losses:",losses) # shape=(?,)
            loss=tf.reduce_mean(losses)#print("2.loss.loss:", loss) #shape=()
            l2_losses = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'bias' not in v.name]) * l2_lambda
            loss=loss+l2_losses
        return loss
    
    #NCE损失
    def loss_nce(self,l2_lambda=0.0001): #0.0001-->0.001
        """calculate loss using (NCE)cross entropy here"""
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        if self.is_training: #training
            #labels=tf.reshape(self.input_y,[-1])               #[batch_size,1]------>[batch_size,]
            labels=tf.expand_dims(self.input_y,1)                   #[batch_size,]----->[batch_size,1]
            loss = tf.reduce_mean( #inputs: A `Tensor` of shape `[batch_size, dim]`.  The forward activations of the input network.
                tf.nn.nce_loss(weights=tf.transpose(self.W_projection),#[hidden_size*2, num_classes]--->[num_classes,hidden_size*2]. nce_weights:A `Tensor` of shape `[num_classes, dim].O.K.
                               biases=self.b_projection,                 #[label_size]. nce_biases:A `Tensor` of shape `[num_classes]`.
                               labels=labels,                 #[batch_size,1]. train_labels, # A `Tensor` of type `int64` and shape `[batch_size,num_true]`. The target classes.
                               inputs=self.output_rnn_last,# [batch_size,hidden_size*2] #A `Tensor` of shape `[batch_size, dim]`.  The forward activations of the input network.
                               num_sampled=self.num_sampled,  #scalar. 100
                               num_classes=self.num_classes,partition_strategy="div"))  #scalar. 1999
        l2_losses = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'bias' not in v.name]) * l2_lambda
        loss = loss + l2_losses
        return loss

    def train(self):
        """based on the loss, use SGD to update parameter"""
        learning_rate = tf.train.exponential_decay(self.learning_rate, self.global_step, self.decay_steps,self.decay_rate, staircase=True)
        train_op = tf.contrib.layers.optimize_loss(self.loss_val, global_step=self.global_step,learning_rate=learning_rate, optimizer="Adam")
        return train_op

#分类测试 
def test():

    num_classes=10
    learning_rate=0.001
    batch_size=8
    decay_steps=1000
    decay_rate=0.9
    sequence_length=5
    vocab_size=10000
    embed_size=100
    is_training=True
    dropout_keep_prob=1#0.5
    textRNN=TextRNN(num_classes, learning_rate, batch_size, decay_steps, decay_rate,sequence_length,vocab_size,embed_size,is_training)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(100):
            input_x=np.zeros((batch_size,sequence_length)) #[None, self.sequence_length]
            input_y=input_y=np.array([1,0,1,1,1,2,1,1]) #np.zeros((batch_size),dtype=np.int32) #[None, self.sequence_length]
            loss,acc,predict,_=sess.run([textRNN.loss_val,textRNN.accuracy,textRNN.predictions,textRNN.train_op],feed_dict={textRNN.input_x:input_x,textRNN.input_y:input_y,textRNN.dropout_keep_prob:dropout_keep_prob})
            print("loss:",loss,"acc:",acc,"label:",input_y,"prediction:",predict)
test()


outputs:===> (<tf.Tensor 'bidirectional_rnn/fw/fw/transpose_1:0' shape=(?, 5, 100) dtype=float32>, <tf.Tensor 'ReverseV2:0' shape=(?, 5, 100) dtype=float32>)
loss: 2.8903913 acc: 0.125 label: [1 0 1 1 1 2 1 1] prediction: [2 2 2 2 2 2 2 2]
loss: 2.721138 acc: 0.125 label: [1 0 1 1 1 2 1 1] prediction: [2 2 2 2 2 2 2 2]
loss: 2.5252109 acc: 0.125 label: [1 0 1 1 1 2 1 1] prediction: [2 2 2 2 2 2 2 2]
loss: 2.2736619 acc: 0.75 label: [1 0 1 1 1 2 1 1] prediction: [1 1 1 1 1 1 1 1]
loss: 1.9544692 acc: 0.75 label: [1 0 1 1 1 2 1 1] prediction: [1 1 1 1 1 1 1 1]
loss: 1.6184561 acc: 0.75 label: [1 0 1 1 1 2 1 1] prediction: [1 1 1 1 1 1 1 1]
loss: 1.4595925 acc: 0.75 label: [1 0 1 1 1 2 1 1] prediction: [1 1 1 1 1 1 1 1]
loss: 1.5684565 acc: 0.75 label: [1 0 1 1 1 2 1 1] prediction: [1 1 1 1 1 1 1 1]
loss: 1.618169 acc: 0.75 label: [1 0 1 1 1 2 1 1] prediction: [1 1 1 1 1 1 1 1]
loss: 1.5419416 acc: 0.75 label: [1 0 1 1 1 2 1 1] prediction: [1 1 1 1 1 1 1 1]
loss: 1.4079881 acc: 0.75 label

In [1]:
'''
* 利用RCNN模型来进行文本分类。
'''

import tensorflow as tf

#RCNN实现
class TextRCNN:
    def __init__(self, sequence_length, num_classes, vocab_size, word_embedding_size, context_embedding_size,
                 cell_type, hidden_size, l2_reg_lambda=0.0):
        # 初始化输入x,y,  dropout
        self.input_text = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_text')
        self.input_y = tf.placeholder(tf.float32, shape=[None, num_classes], name='input_y')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')

        l2_loss = tf.constant(0.0)
        text_length = self._length(self.input_text)

        # 输入层【用embedding_lookup】
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W_text = tf.Variable(tf.random_uniform([vocab_size, word_embedding_size], -1.0, 1.0), name="W_text")
            self.embedded_chars = tf.nn.embedding_lookup(self.W_text, self.input_text)

        # 【双向（左、右）循环结构】【使用tf的bidirectional_dynamic_rnn】
        with tf.name_scope("bi-rnn"):
            fw_cell = self._get_cell(context_embedding_size, cell_type)
            fw_cell = tf.nn.rnn_cell.DropoutWrapper(fw_cell, output_keep_prob=self.dropout_keep_prob)
            bw_cell = self._get_cell(context_embedding_size, cell_type)
            bw_cell = tf.nn.rnn_cell.DropoutWrapper(bw_cell, output_keep_prob=self.dropout_keep_prob)
            (self.output_fw, self.output_bw), states = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_cell,
                                                                                       cell_bw=bw_cell,

        with tf.name_scope("context"):
            shape = [tf.shape(self.output_fw)[0], 1, tf.shape(self.output_fw)[2]]
            self.c_left = tf.concat([tf.zeros(shape), self.output_fw[:, :-1]], axis=1, name="context_left")
            self.c_right = tf.concat([self.output_bw[:, 1:], tf.zeros(shape)], axis=1, name="context_right")

        with tf.name_scope("word-representation"):
            self.x = tf.concat([self.c_left, self.embedded_chars, self.c_right], axis=2, name="x")
            embedding_size = 2*context_embedding_size + word_embedding_size

        with tf.name_scope("text-representation"):
            W2 = tf.Variable(tf.random_uniform([embedding_size, hidden_size], -1.0, 1.0), name="W2")
            b2 = tf.Variable(tf.constant(0.1, shape=[hidden_size]), name="b2")
            self.y2 = tf.tanh(tf.einsum('aij,jk->aik', self.x, W2) + b2)

        with tf.name_scope("max-pooling"):
            self.y3 = tf.reduce_max(self.y2, axis=1)

        with tf.name_scope("output"):
            W4 = tf.get_variable("W4", shape=[hidden_size, num_classes], initializer=tf.contrib.layers.xavier_initializer())
            b4 = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b4")
            l2_loss += tf.nn.l2_loss(W4)
            l2_loss += tf.nn.l2_loss(b4)
            self.logits = tf.nn.xw_plus_b(self.y3, W4, b4, name="logits")
            self.predictions = tf.argmax(self.logits, 1, name="predictions")

        # 计算平均交叉熵损失
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # 精确度
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, axis=1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name="accuracy")

    @staticmethod
    def _get_cell(hidden_size, cell_type):
        if cell_type == "vanilla":
            return tf.nn.rnn_cell.BasicRNNCell(hidden_size)
        elif cell_type == "lstm":
            return tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
        elif cell_type == "gru":
            return tf.nn.rnn_cell.GRUCell(hidden_size)
        else:
            print("ERROR: '" + cell_type + "' is a wrong cell type !!!")
            return None

    # 句子长度
    @staticmethod
    def _length(seq):
        relevant = tf.sign(tf.abs(seq))
        length = tf.reduce_sum(relevant, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        return length

    # 提取每个句子最后单元的输出
    # 如：) The movie is good -> length = 4
    #     output = [ [1.314, -3.32, ..., 0.98]
    #                [0.287, -0.50, ..., 1.55]
    #                [2.194, -2.12, ..., 0.63]
    #                [1.938, -1.88, ..., 1.31]
    #                [  0.0,   0.0, ...,  0.0]
    #                ...
    #                [  0.0,   0.0, ...,  0.0] ]
    #     需要单元的第4个输出来提出它
    @staticmethod
    def last_relevant(seq, length):
        batch_size = tf.shape(seq)[0]
        max_length = int(seq.get_shape()[1])
        input_size = int(seq.get_shape()[2])
        index = tf.range(0, batch_size) * max_length + (length - 1)
        flat = tf.reshape(seq, [-1, input_size])
        return tf.gather(flat, index)




In [ ]:
'''

RNN Recurrent（回路） Neural Network 循环神经网络。

RNN的结构。循环神经网络的提出背景、优缺点。着重学习RNN的反向传播、RNN出现的问题（梯度问题、长期依赖问题）、BPTT算法。

RNN的结构：
传统神经网络有输入层、一个或多个隐藏层、输出层；
相比之下，RNN循环神经网络，在隐层里多了一步递归，
将结果输出一下层后，中间信息保存在记忆单元c中，
将本次记忆的信息加在下一次传播过来的wx里，一起传给下一层。
记忆单元c在整个传播过程中也是跟着每步更新的。

提出背景：
是为了解决前后传递的数据之间无关联的问题，
比如NLP里，输入一行文字预测下一词时，我们前面如果有相关上下文背景会提高预测精度，减少运算。

优点：
RNN结构让机器学到了语序和上下文关联。比如做一个二制进加法操作，
机器加本位的同时可以用前一位的记忆，学习到进位法则，自己学会运算。

RNN出现的问题、缺点：
记忆单元累积到最终输出时可能是大量的,有长期依赖问题，
这些内容有一些距离远的失效信息增加了信息冗余；
且在反向传播时，每一个记忆分支都要依次算梯度，增加计算复杂度，同时会出现梯度消失，
之后的权重都变为，以后不会再被用算到；
过多很小数累乘的结果本身也是约等于的，也能导致模型不能继续下去。

RNN的反向传播：
反身传播与传统神经网络一样，从结果值中取到值逐层逆向卷积算下降梯度，求导，取最小值，
不同的是除了和正常传递层计算wx+b以外，还要和有存储信息的，记忆单元分支算梯度，
合在一起链式法则向前回传。

BPTT算法：
Back Propagation Trough Time 基于时间的反向传播算法，

双向RNN：
双向RNN认为输出不仅依赖于序列之前的元素，也跟它之后的元素有关，
这在序列挖掘中也是很常见的。

递归神经网络：
recursive neural network 递归神经网络是树状阶层拓扑结构，
是RNN循环神经网络的升级，递归神经网络的每个父节点如果只有一个子节点时，
则其结构等价全连接的循环神经网络。
递归神经网络可以引入门控机制，设置记忆信息的阈值取舍，学习长距离依赖。
灵活的拓扑还有权重共享，适用于包含结构关系的机器学习、深度学习任务，
在NLP领域有重要应用。

LSTM：
Long Short Term Memory，可以解决上面出现的过远的信息长期依赖问题。又能学到学习到长期依赖关系。
普通的RNN是每个模块内都加了一个tanh记忆层。
LSTM每个循环的模块内又有4层结构:3个sigmoid层（forget，input，output），1个tanh层。
sigmoid层用于做门单元，通过阈值控制参数来决定什么样的信息保留，超过多久之前的信息不保留之类的。

GRU：
Gated Recurrent Unit，是LSTM的一个变体。
GRU只有两个门（update和reset），LSTM有三个门（forget，input，output）。
LSTM的input gate负责控制new memory，输入信息，
forget gate负责控制上一轮的memory，长期信息，
output gate对前两者的激活信息进行控制，输出h，当前隐藏层信息。
GRU的reset gate直接在new memory处对上一层的隐层信息进行处理，控制h(t-1)的信息存在，
所以这里update gate是对输入信息的控制得到当前层隐层状态。
GRU 参数相对少更容易收敛，但是在数据集较大的情况下，LSTM性能更好。
LSTM还有许多变体，但不管是何种变体,，重点在于额外的门控机制是如何设计，用以控制梯度信息传播从而缓解梯度消失现象。

针对梯度消失（LSTM等其他门控RNN）、梯度爆炸（梯度截断）的解决方案：
梯度消失的解决方案：
可以用LSTM门控RNN缓解梯度消失，控制信息长期依赖；
初始化矩阵时不用随机矩阵，用单位矩阵；
激活函数用Relu，PReLU；

梯度爆炸（梯度截断）的解决方案：
梯度爆炸是在断涯式数据处求梯度不稳定的现象。
RNN中可以用梯度裁剪gradient clipping避免，
在迭代中各权重的梯度平方和如果大于某个阈值，为了避免权重的变化值太大，
求一个缩放因子（阈值/平方和），将所有梯度乘以这个因子；
添加正则项。

Memory Network：
Memory Network，记忆网络出现之前，大多数机器学习的模型都缺乏可以读取和写入外部知识的组件，
例如，给定一系列事实或故事，然后要求回答关于该主题的问题。
RNN虽然可以被训练在阅读了一串文字之后用来预测下一个输出，但记忆太小，不能精确记住过去的事实。
一个Memory Network由一个记忆数组m（一个向量的数组或者一个字符串数组，index by i）
和四个组件（输入I，泛化G，输出O，回答R）组成。
I :（输入特征映射） - 将输入转换为记忆网络内部特征的表示。给定输入x，可以是字符、单词、句子等不同的粒度，通过I(x)得到记忆网络内部的特征。
G :（更新记忆） - 使用新的输入更新记忆数组m。 mi=G(mi,I(x),m)
O：（输出） - 在记忆数组m更新完以后，就可以将输入和记忆单元联系起来，根据输入选择与之相关的记忆单元。 o=O(I(x),m)
R：（输出回答） - 得到了输入编码向量I(x)，记忆数组m和需要的支持事实，就可以根据问题来得到需要的答案了。

参考：https://www.cnblogs.com/zhangchaoyang/articles/6684906.html
https://blog.csdn.net/sinat_33741547/article/details/82821782
'''

In [ ]:
'''
Text-RNN的原理。
基于RNN的文本分类模型。
TextCNN擅长捕获短的序列信息，
TextRNN擅长捕获更长的序列信息。
具体到文本分类任务中，BiLSTM可以理解为可以捕获变长且双向的N-Gram信息。
文本分类也可以理解为一种特殊的Seq2Seq模型
Seq2Seq模型的标配是引入了注意力机制[Attention]能够很好的给出每个词对结果的贡献程度，
注意力机制的引入，可以在某种程度上提高深度学习文本分类模型的可解释性。
TextRNN的结构非常灵活，可以任意改变。
比如把LSTM单元替换为GRU单元，把双向改为单向，添加dropout或BatchNormalization以及再多堆叠一层等等。
TextRNN在文本分类任务上的效果非常好，与TextCNN不相上下，但RNN的训练速度相对偏慢，一般2层就已经足够多了。

Recurrent Convolutional Neural Networks（RCNN）原理:
双向循环神经网络，RCNN可以较均匀的利用单词的上下文信息，
既可以缓解在RNN中后面的单词比前面的单词影响力更大的缺点，
也不需要像CNN一样需要通过窗口大小来设定对上下文的依赖长度。
每一个单词的embedding方式主要有3个部分concat组成：
left context ;
单词本身的embedding;
righ context，
w代表单词。
单词本身的embedding，使用了Skip-gram的方法进行预训练。
工作流程：
假定单词w先经过1层双向LSTM，
该词的左侧的词正向输入进去得到一个词向量，
该词的右侧反向输入进去得到一个词向量。
再结合该词的词向量，生成一个 1 * 3k 的向量。
再经过全连接层，tanh为非线性函数，得到y2。
再经过最大池化层，得出最大化向量y3.
再经过全连接层，sigmod为非线性函数，得到最终的多分类。

'''